In [1]:
#하시고 나서 requirments.txt 다시 안돌려주시면 제출 에러납니다.
!pip install scikit-image --upgrade

  Using cached scikit_image-0.18.1-cp37-cp37m-manylinux1_x86_64.whl (29.2 MB)
  Using cached numpy-1.20.3-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.3 MB)
ERROR: sagemaker-training 3.9.2 has requirement scipy>=1.2.2, but you'll have scipy 1.2.0 which is incompatible.
  Attempting uninstall: numpy
    Found existing installation: numpy 1.15.4
    Uninstalling numpy-1.15.4:
      Successfully uninstalled numpy-1.15.4
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.14.1
    Uninstalling scikit-image-0.14.1:
      Successfully uninstalled scikit-image-0.14.1


In [1]:
from glob import glob
import matplotlib.pyplot as plt

train_dir = 'input/data/train_dataset/'

#필요한 파일 로드

img_list = sorted(list(glob(train_dir + 'images/*')))

def claheCVT(own_img) :
    lab = cv2.cvtColor(own_img, cv2.COLOR_BGR2LAB)
    lab_planes = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit = 2.0, tileGridSize = (8, 8))
    lab_planes[0] = clahe.apply(lab_planes[0])
    lab = cv2.merge(lab_planes)
    cla_img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
        
    return cla_img

#이미지 로드 및 AUG 수행
for num, img in enumerate(img_list) :
    img_name = 'xx' + img[-15:]
    
    #이미지 로드
    img = cv2.imread(img)
    img_own = img
    img_cl1 = claheCVT(img_own)
    img_cl2 = claheCVT(img_cl1)
    
    fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(6, 6))
    ax1.imshow(img_own)
    ax2.imshow(img_cl1)
    ax3.imshow(img_cl2)
    plt.show()
    
    if num == 10 : break


Bad key "text.kerning_factor" on line 4 in
/opt/conda/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


NameError: name 'cv2' is not defined

In [36]:
from glob import glob
import random

train_dir = 'input/data/train_dataset/'

#필요한 파일 로드

img_list = sorted(list(glob(train_dir + 'images/*')))

f = open(train_dir + 'gt.txt', 'r')
gt = f.readlines()
f.close()

f = open(train_dir + 'level.txt', 'r')
lv = f.readlines()
f.close()

f = open(train_dir + 'source.txt', 'r')
sc = f.readlines()
f.close()


print(len(img_list), len(gt), len(lv), len(sc))

150000 150000 150000 150000


In [37]:
#랜덤하게 1,000개 추출
#직접 만들고 싶으시면 seed 값을 변경해가며 시도하시면 다양한 데이터셋을 만드실 수 있습니다.

SEED = 59
random.seed(SEED)
aug_list = list(range(0, 100000))
random.shuffle(aug_list)
#1000개만 생성
aug_list = sorted(aug_list[:50000])


img_list_tmp = list()

for i in aug_list :
    img_list_tmp.append(img_list[i])
    gt.append('xx' + gt[i])
    lv.append('xx' + lv[i])
    sc.append('xx' + sc[i])
    
img_list = img_list_tmp
print(len(img_list), len(gt), len(lv), len(sc))

50000 200000 200000 200000


In [38]:
import torch
from PIL import Image
import numpy
from torchvision import transforms
import cv2, os
import matplotlib.pyplot as plt
import numpy as np
import albumentations as A
from tqdm import tqdm

#디렉터리 만들기
dir_path = "input/data/train_dataset_cpy"
if not(os.path.isdir(dir_path)) : 
    os.makedirs(dir_path)
    os.makedirs(dir_path + "/images")

#이미지 로드 및 AUG 수행
for num, img in enumerate(img_list) :
    img_name = 'xx' + img[-15:]
    
    #이미지 로드
    img = cv2.imread(img)
    img_own = img
    
    #numpy => Tensor
    img = transforms.ToPILImage()(img) #.ToTensor()(img)
    
    #Rotate
    #가로가 길면 세로로,
    #세로가 길면 가로로
    # 1/2 확률로 변환
    isChange = random.choice([True, False])
    if isChange :
        if img.size[0] > img.size[1] : img = transforms.RandomRotation((90, 90), expand = True)(img)
        else : img = transforms.RandomRotation((-90, -90), expand = True)(img)
    
    #RGB 재조정
    img = transforms.ColorJitter(brightness = 0.5, contrast = 0.5, saturation = 0.5, hue = 0.5) (img)
    
    #A 사용을 위해 np 변환
    img = np.array(img)
    
    #Distortion
    img = A.OpticalDistortion(p=1, distort_limit = (-0.2, 0.2), shift_limit = (-0.1, 0.1))(image = img)["image"]
    img = A.GridDistortion(p=1, distort_limit=0.02)(image = img)["image"]
    
    #2차 RGB 재조정
    img = A.RGBShift(p=1)(image=img)["image"]
    
    #CLAHE로 선명도 개선
    #너무 잘보이게 바꾸는 경향이 있어 1/3 확률로 적용
    img = A.CLAHE(p=0.33)(image = img)["image"]
    
    #상황에 따른 노이즈 추가
    
    #종이에 볼펜으로 점 찍은 상황
    img = A.CoarseDropout(p=0.8, max_holes = 20, max_width=1)(image = img)["image"]
    #카메라가 안좋은 상황
    #img = A.RandomFog(p=0.8, fog_coef_upper=0.3)(image = img)["image"]
    #낙서 혹은 형광펜
    img = A.RandomShadow(p=0.8, num_shadows_upper=1, shadow_dimension = 4)(image = img)["image"]
    #카메라가 심하게 흔들린 상황
    img = A.Blur (p=0.8, blur_limit=5)(image = img)["image"]
    
    #저장asd
    img = Image.fromarray(img)
    img.save(dir_path + "/images/" + img_name)
    
    if num % 1000 == 0 : print(num, '/', len(img_list))
    '''
    #1/4 확률로 시각화
    willShow = random.choice([True, False, False, False])
    if willShow :
        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(6, 6))
        ax1.imshow(img_own)
        ax2.imshow(img)
        plt.show()
    '''

TypeError: 'int' object is not subscriptable

In [31]:
#하지마 하면 안돼 제발 하지마 알았지 하지마

with open(dir_path + "/input.txt", 'w') as f:
    f.writelines(gt)
        
with open(dir_path + "/gt.txt", "w") as f:
    f.writelines(gt)
    
with open(dir_path + "/level.txt", "w") as f:
    f.writelines(lv)
    
with open(dir_path + "/source.txt", "w") as f :
    f.writelines(sc)

In [21]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          99G   28G   66G  30% /
tmpfs            64M     0   64M   0% /dev
tmpfs            45G     0   45G   0% /sys/fs/cgroup
shm             1.0G  4.0K  1.0G   1% /dev/shm
/dev/xvdb1       99G   28G   66G  30% /etc/hosts
tmpfs            45G   12K   45G   1% /proc/driver/nvidia
/dev/xvda1       48G   29G   16G  65% /usr/bin/nvidia-smi
tmpfs           8.9G   17M  8.9G   1% /run/nvidia-persistenced/socket
udev             45G     0   45G   0% /dev/nvidia0
tmpfs            45G     0   45G   0% /proc/acpi
tmpfs            45G     0   45G   0% /proc/scsi
tmpfs            45G     0   45G   0% /sys/firmware
